In [1]:
import pandas as pd
import numpy as np
import os
import sqlite3
import shutil
import datetime
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import matplotlib.ticker as tick
from IPython.display import HTML, display, Markdown
import tabulate
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Layout
import graphviz
from GraphVizUtil import *
from GraphVizFormats import *
import warnings
warnings.filterwarnings('ignore')
import seaborn as sb
sb.set(style='darkgrid', font_scale=1.2)

def filter_descriptions(tech_comm_desc):
    try:
        tech_comm_desc = tech_comm_desc.values[0][0].replace('#', '').replace('"','').replace("\n",'').strip()
    except:
        tech_comm_desc = 'No description provided'
    return tech_comm_desc

def create_args_flowd(df_graph):
    nodes, tech, ltech, to_tech, from_tech = set(), set(), set(), set(), set()
    for ind,row in df_graph.iterrows():
        #descriptions:
        input_comm_des = filter_descriptions(pd.read_sql("SELECT comm_desc FROM commodities WHERE comm_name='" + row['input_comm'] + "'", con))
        output_comm_des = filter_descriptions(pd.read_sql("SELECT comm_desc FROM commodities WHERE comm_name='" + row['output_comm'] + "'", con))
        tech_des = filter_descriptions(pd.read_sql("SELECT tech_desc FROM technologies WHERE tech='" + row['tech'] + "'", con))

        if 'ethos' in row['input_comm']:
            ltech.add('"' + row['tech'] + '"' +  ' [tooltip = "' + tech_des + '"]')
        else :
            nodes.add('"' + row['input_comm'] + '"' +  ' [tooltip = "' + input_comm_des + '"]')
        nodes.add('"' + row['output_comm'] + '"' +  ' [tooltip = "' + output_comm_des + '"]')
        tech.add('"' + row['tech'] + '"' +  ' [tooltip = "' + tech_des + '"]')

        if row['input_comm'] != 'ethos':
            to_tech.add('"%s"' % row['input_comm'] + '\t->\t"%s"' % row['tech']) 
        from_tech.add('"%s"' % row['tech'] + '\t->\t"%s"' % row['output_comm'])
    args = dict(
    enodes = "".join('%s;\n\t\t' % x for x in nodes),
    tnodes = "".join('%s;\n\t\t' % x for x in tech),
    iedges = "".join('%s;\n\t\t' % x for x in to_tech),
    oedges = "".join('%s;\n\t\t' % x for x in from_tech),
    snodes = ";".join('%s' %x for x in ltech),
    )
    return args

def return_format_colors():
    colors = {}
    colors.update(getColorConfig(False))
    return colors, quick_run_dot_fmt

def return_flowd_table(final_dem, level=1):
    df = pd.read_sql("SELECT * FROM Efficiency", con)
    df_sel = df[df['output_comm']==final_dem]
    if len(df_sel)==0:
        df_sel = df[df['tech']==final_dem]
    inputs = df_sel['input_comm'].unique()
    iterval=0
    if level!=0:
        while len(inputs)>0:
            df_append = df[df['output_comm'].isin(inputs)]
            df_sel = pd.concat([df_sel, df_append])
            inputs = df_append['input_comm'].unique()
            iterval+=1
            if iterval>level-1:
                break
    df_graph = df_sel[['input_comm', 'tech', 'output_comm']].drop_duplicates()
    return df_graph

def return_flowd_table_fwds(final_dem):
    df = pd.read_sql("SELECT * FROM Efficiency", con)
    df_sel = df[df['output_comm']==final_dem]
    if len(df_sel)==0:
        df_sel = df[df['tech']==final_dem]
    inputs = df_sel['input_comm'].unique()
    outputs = df_sel['output_comm'].unique()

    iterval=0
    while len(inputs)>0:
        df_append = df[df['output_comm'].isin(inputs)]
        df_sel = pd.concat([df_sel, df_append])
        inputs = df_append['input_comm'].unique()
        iterval+=1
        if iterval>2:
            break
    iterval=0
    while len(outputs)>0:
        df_append = df[df['input_comm'].isin(outputs)]
        df_sel = pd.concat([df_sel, df_append])
        outputs = df_append['output_comm'].unique()
        iterval+=1
        if iterval>=0:
            break

    df_graph = df_sel[['input_comm', 'tech', 'output_comm']].drop_duplicates()
    return df_graph

con = sqlite3.connect(r'../US_9R_8D.sqlite') #change path to database
cur = con.cursor()   
con.text_factory = str 

def controls_rows(w):
    controls = HBox(w.children[:-1], layout = Layout(flex_flow='row wrap', width='max-content'))
    output = w.children[-1]
    display(VBox([controls, output],  layout = Layout(flex_flow='columns wrap', width='max-content', size=10)))
    
#https://www.eia.gov/energyexplained/units-and-calculators/
btu_per_oil_bbl = 5698000
PJ_per_oil_bbl = btu_per_oil_bbl*1.05506e-12

# Introduction
The fuel supply sector consists of commodities and technologies that provide refined petroleum products, coal, natural gas and biomass resources to the energy modeling system. These resources are discussed separately in the sections that follow.

# Fossil fuels

We use exogenous, regional petroleum, coal and natural gas fuel prices from the [US EIA Annual Energy Outlook 2022](https://www.eia.gov/outlooks/aeo/) as input to the OEO scenarios. For the No New Policy scenario, we use prices from the AEO's Reference case. For all other scenarios (States Action, COP26 and Net Zero) we use the AEO's Low Oil price case, consistent with expected price impacts due to reductions in petroleum demand. Fuel prices consistent with the different end-use sectors are used. For example, we use specific projections for gasoline supplied to the CA region in the transport sector. Since fuel supply is modeled as exogenous prices with no resource constraints, we do not consider the inter-regional transport of these fuels.

We consider three types of coal (lignite, bituminous and sub-bituminous), although they currently have the same techno-economic and environmental parameters in the database. This is a relic from a previous version of the database and we plan to condense coal to a single commodity in future database iterations. 

In addition, we consider a range of petroleum fuels such as gasoline, diesel, resdidual and distillate fuel oil, jet fuel and marine gas oil

<!-- Fuels to specific sectors are prefixed differently, as shown in the table below.

| Sector | Prefix | 
| ---- | ------------- |
| Electric | ELC | 
| Industrial | IND | 
| Transport | TRN | 
| Residential | RES | 
| Commercial | COM |  -->

The widget below can be used to explore fossil fuel prices supplied across different regions and energy sectors within the OEO.

In [2]:
def show_cost_variable(con):
    
    sector_map = dict()
    sector_map['IMPTRN'] = 'Transport: '
    sector_map['IMPELC'] = 'Electric: '
    sector_map['IMPIND'] = 'Industrial: '
    sector_map['IMPCOM'] = 'Commercial: '
    sector_map['IMPRES'] = 'Residential: '
    
    fuel_map = dict()
    fuel_map['DISTOIL'] = 'Distillate oil'
    fuel_map['DFO'] = 'Distillate oil'
    fuel_map['JTF'] = 'Jet Fuel'
    fuel_map['RFO'] = 'Residual fuel oil '
    fuel_map['NGA'] = 'Natural gas'
    fuel_map[': NG'] = ': Natural gas'
    fuel_map['_S3'] = ''
    fuel_map[': COAL'] = ': Coal'
    fuel_map[': COAS'] = ': Coal'
    fuel_map[': COAB'] = ': Coal'
    fuel_map['URN'] = 'Nuclear'
    fuel_map['GSL'] = 'Gasoline'
    fuel_map['GAS'] = 'Gasoline'
    fuel_map['MGO'] = 'Marine gas oil'
    fuel_map['GAS'] = 'Gasoline'
    fuel_map['OTH'] = 'Other'

    query = "SELECT regions, vintage, periods, tech, cost_variable FROM CostVariable WHERE tech LIKE 'IMP%'\
     AND tech NOT LIKE '%BIO%'"
    df_fuelcosts_nnp = pd.read_sql_query(query, con)
    con_low = sqlite3.connect(r'../US_9R_8D_NZ.sqlite') #change path to database
    df_fuelcosts_nz = pd.read_sql_query(query, con_low)
    
    for keyv in sector_map.keys():
        df_fuelcosts_nnp['tech'] = df_fuelcosts_nnp['tech'].str.replace(keyv, sector_map[keyv])
        df_fuelcosts_nz['tech'] = df_fuelcosts_nz['tech'].str.replace(keyv, sector_map[keyv])
    for keyv in fuel_map.keys():
        df_fuelcosts_nnp['tech'] = df_fuelcosts_nnp['tech'].str.replace(keyv, fuel_map[keyv])
        df_fuelcosts_nz['tech'] = df_fuelcosts_nz['tech'].str.replace(keyv, fuel_map[keyv])

    techs = df_fuelcosts_nnp['tech'].unique()
    regions = df_fuelcosts_nnp['regions'].unique()
    
    def filter_tech(tech ='', region=''):
        df_fuelcosts_nnp_sel = df_fuelcosts_nnp[(df_fuelcosts_nnp['tech'].str.contains(tech)) &\
                                               (df_fuelcosts_nnp['regions']==region)]
        df_fuelcosts_nz_sel = df_fuelcosts_nz[(df_fuelcosts_nz['tech'].str.contains(tech)) &\
                                             (df_fuelcosts_nz['regions']==region)]
        
        df_fuelcosts_nz_sel = df_fuelcosts_nz_sel.drop_duplicates()
        df_fuelcosts_nnp_sel = df_fuelcosts_nnp_sel.drop_duplicates()

        fig, ax = plt.subplots(figsize=(10,6))
        plt.plot(df_fuelcosts_nnp_sel['periods'], df_fuelcosts_nnp_sel['cost_variable'], \
                    label='No New Policy')
        plt.plot(df_fuelcosts_nz_sel['periods'], df_fuelcosts_nz_sel['cost_variable'], \
                    label='Net Zero')
        plt.legend()
        plt.ylabel('Variable costs ($M/PJ)')

    w1 = widgets.Select(options=techs)
    w3 = widgets.Select(options=regions)
    w = widgets.interactive(filter_tech, tech=w1, region=w3)


    controls_rows(w)

show_cost_variable(con)



# Biomass

Unlike coal, oil, and natural gas, where we assume exogenous prices that varies with time period, we construct supply curves of domestic biomass resources given their limited supply. We use the [Billion Ton Report](https://www.energy.gov/eere/bioenergy/2016-billion-ton-report) produced by Oak Ridge National Laboratory and process the publicly available raw data into a set of supply curves.

As described below, these biomass resources can be used in electric generation, the production of biofuels, including ethanol, jet fuel, biodiesel and the production of hydrogen. These conversion pathways may include carbon capture and storage (BECCS) as well. Biofuel conversion processes represented in the database include ethanol production via corn and corn stover, transesterification to produce biodiesel, HEFA to produce jet fuel, and Fischer-Tropsch to produce biodiesel and jet fuel. 

## Biomass Resources

Biomass supply curves are derived by largely following the approach in [Net Zero America (NZA)](https://netzeroamerica.princeton.edu/img/NZA%20Annex%20H%20-%20Bioenergy%20transition.pdf). Data from three sources are taken to construct the supply curves used in NZA and consequently the Open Energy Outlook (OEO): 
* [Billion Ton Study (BTS16)](https://bioenergykdf.net/2016-billion-ton-report) from Oak Ridge National Laboratory
* [Conservation Reserve Program Land data](https://www.fsa.usda.gov/programs-and-services/conservation-programs/reports-and-statistics/conservation-reserve-program-statistics/index) from USDA
* [Corn Ethanol Land data](https://quickstats.nass.usda.gov/#B9AA9E09-0486-3B84-8A9A-0C28BB4242D3) from USDA

BTS16 data includes production in dry tons and biomass prices in $2016/dry tons for energy crops like corn and soy, agricultural residues, woody biomass and wastes (raw data available [here](https://bioenergykdf.net/sites/default/files/BillionTonDownloads/billionton_county_all.zip)). Perennial grasses were modeled using acreage by U.S. State estimates from the conservation reserve program and corn ethanol land data. The acreage data from these two resources are subsequently represented on a dry ton basis using the yield data provided in NZA (Annex H Section 2.3 and Table 3). An energy density of 19.8 GJ/dry metric ton of biomass was assumed for representation on an energy basis (NZA Annex H). 

From BTS16, estimates from 2040 are used to build future projections of biomass potential while 2020 data is used to describe currently utilized resources. BTS16 projections end in 2040 and like in NZA we assume that the 2040 levels are the maximum available in 2040 and beyond. Table 1 in [NZA's Annex H](https://netzeroamerica.princeton.edu/img/NZA%20Annex%20H%20-%20Bioenergy%20transition.pdf) lists the subset of resources that are considered in their biomass representation from BTS16. Supply curves for OEO are constructed using the 'delimited' scenario by selecting only the respective feedstocks listed in this table. This scenario reflects the biomass potential projected from agriculture residues, woody residues, wastes and energy grasses grown on lands from conservation research program lands and from growing corn for ethanol. 

The steps taken from raw production/price data to final OEO supply curve representations are detailed in the *BiomassSupplyCurves* Jupyter notebook. Briefly, state level production data was aggregated by binning the biomass feedstocks listed in Table 1 into the following broad categories: 1) waste 2) herbaceous and 3) woody biomass resources along with specific representations for corn, corn stover and soybeans. Production data was aggregated by the OEO regions. Finally, supply curves were constructed as the price versus quantity relationship for each of the aggregated resource categories (as well as corn, corn stover and soybeans).

Regional biomass supply curves can be viewed using the tool below. 

In [3]:
def bio_supply_curve(con):
    query = "SELECT * FROM MaxActivity WHERE tech LIKE '%_STEP%' AND periods = 2050"
    df_maxact = pd.read_sql_query(query, con)

    query = "SELECT * FROM CostVariable WHERE tech LIKE '%_STEP%' AND periods = 2050"
    df_stepcost = pd.read_sql_query(query, con)

    df_supply_curve_bio = pd.merge(df_maxact, df_stepcost, on =['regions','tech', 'periods'])

    regions = ['All'] + list(set(df_supply_curve_bio['regions']))    

    cb = dict()
    cb['Woody'] = 'indianred'
    cb['Herbaceous'] = 'orange'
    cb['Waste'] = 'g'
    cb['Corn'] = 'grey'
    cb['Soybeans'] = 'b'
    cb['Corn stover'] = 'red'

    def select_region(region):
        if region !='All':
            df_supply_curve_bio_plot = df_supply_curve_bio[df_supply_curve_bio['regions']==region]
        else:
            df_supply_curve_bio_plot = df_supply_curve_bio

        df_supply_curve_bio_plot = df_supply_curve_bio_plot.sort_values(by='cost_variable')
        df_supply_curve_bio_plot['Crop Form'] = [x[0] for x in df_supply_curve_bio_plot['tech'].str.split('_')]
        df_supply_curve_bio_plot['Cumulative_Production'] = df_supply_curve_bio_plot['maxact'].cumsum()

        fig, ax = plt.subplots(figsize=(10,6))
        start = 0
        for ind, row in df_supply_curve_bio_plot.iterrows():
            x = start + (row['Cumulative_Production']-start)/2
            plt.bar(x, row['cost_variable'] , width=row['Cumulative_Production']-start, color=cb[row['Crop Form']], \
                    label=row['Crop Form'], linewidth = 1, edgecolor= cb[row['Crop Form']])
            start = row['Cumulative_Production']

        plt.xlabel('Biomass resource (PJ)')
        plt.ylabel('Biomass price ($M/PJ)')
        plt.legend()
        handles, labels = ax.get_legend_handles_labels()
        unique = [(h, l) for i, (h, l) in enumerate(zip(handles, labels))
                  if l not in labels[:i]]
        plt.legend(*zip(*unique))
        ax.get_xaxis().set_major_formatter(
        tick.FuncFormatter(lambda x, p: format(int(x), ',')))

    w3 = widgets.Select(options=regions)
    w = widgets.interactive(select_region, region=w3)


    controls_rows(w)
bio_supply_curve(con)


## Biomass emissions factors
All biomass (except corn) is assumed to have an upstream emissions factor that characterizes life cycle steps such as feedstock production, transport and land-use change - these include corn stover, herbaceous, soybeans, woody and waste biomass. An emissions factor based on the recent [IPCC Working Group III, 6th Assessment Report](https://www.ipcc.ch/working-group/wg3/) (Box 7.7, Fig 1, IAM median value at 200 EJ of biomass utilization globally) of 20 kt CO<sub>2</sub>/PJ is applied to these biomass feedstocks. Combustion emissions for woody, herbaceous and waste feedstocks are assumed to be 93.8 kg CO<sub>2</sub>/MMBtu, based on EPA data. For biofuels produced from corn, life cycle emissions factors are applied to the ethanol production pathway. 

## Biomass in electric sector

The biomass feedstocks described above contribute to three different electric generation technologies, as described below.

### Existing biopower capacity

The technology ("E_BIO_R") represents existing bio-power in the electric sector; approximately 4 GW of capacity exist. *E_BIO_R* can accept herbaceous feedstocks from the supply curves referenced above. Data to characterize the process is drawn from the NREL Annual Technology Baseline (ATB) via PowerGenome.

### New biopower capacity

The technology ("E_BIO_N") represents new bio-power in the electric sector. The underlying technology is assumed to be a fluidized bed reactor, which can accept a wide variety of biomass feedstocks. In the OEO, we assume that *E_BIO_N* can accept herbaceous feedstocks. Data to chacterize the process is drawn from the NREL Annual Technology Baseline via PowerGenome. NREL, in turn, drawns on the EIA's [Assumptions to the Annual Energy Outlook](https://www.eia.gov/outlooks/aeo/assumptions/pdf/electricity.pdf) to chacterize the technical and economic performance of the fluidized bed reactor.

### Biomass energy with carbon capture and storage (BECCS)

Biomass energy with carbon capture and storage (BECCS) is a negative emissions technology, as it captures carbon neutral biomass and geologically sequesters the emissions after combustion. 

We assume that BECCS can make use of woody biomass, agricultural residues, and waste feedstocks. [Emenike et al. (2020)](https://www.sciencedirect.com/science/article/abs/pii/S2213138820300916) indicate that the performance differences between three different feedstock types (white wood, miscanthus, straw) are relatively minor. In addition, costs appear to be driven by differences in feedstock prices and capital costs. While differences in material handling and processing differ by feedstock type, we make the simplifying assumption that those cost differences are within the current range of cost uncertainty. 

The cited range of capital costs for BECCS is quite large. [Krey et al. (2019)](https://www.sciencedirect.com/science/article/pii/S0360544218325039) compare techno-economic assumptions across different integrated assessment models. Appendix C of the paper includes BECCS assumptions for capital cost, fixed operations and maintenance costs, and efficiency, which are drawn from the GCAM, IMAGE, MESSAGEix-GLOBIOM_1.0, POLES MILES, REMIND 1.6 models. We take the average across the models by 5-year time period from 2020-2050.

## Biofuel production

The OEO database represents several pathways to convert biomass feedstocks into transportation fuels. Those pathways are outlined below.

### Ethanol production

As described above, the supply of corn and corn stover is drawn from the Oak Ridge Billion Ton Study. For the corn-to-ethanol conversion process, the technology used is dry milling ("DRY_MILL"), which involves grinding the corn into a fine granular powder, which is then treated with enzymes, and the resulting fermentation produces ethanol. Estimates of efficiency and emissions factors (including life cycle CO<sub>2</sub> emissions) are drawn from the Greenhouse gases, Regulated Emissions, and Energy use in Technologies ([GREET](https://greet.es.anl.gov/)) model developed at Argonne National Laboratory. Capital costs as well as fixed and variable operations and maintenance costs are drawn from [Tao et al. (2014)](https://onlinelibrary.wiley.com/doi/full/10.1002/bbb.1431).


For cellulosic ethanol, capital costs as well as fixed and variable operations and maintenance costs are drawn from [Zhao et al. (2015)](https://www.mdpi.com/1996-1073/8/5/4096). 
Conversion efficiency is obtained from [Luo et al. (2009)](https://www.sciencedirect.com/science/article/abs/pii/S136403210900029X). As with conventional corn ethanol, emissions estimates for cellulosic ethanol are drawn from GREET.

Maximum ethanol consumption levels are based on the land requirements for current ethanol production, plus the addition of minimum estimated residual lands on which biofuel feedstocks could be grown at a minimum yield value ([Cai et al., 2010](https://pubs.acs.org/doi/10.1021/es103338e); [Hay, 2015](https://farm-energy.extension.org/corn-for-biofuel-production/); [National Agricultural Statistics Service, 2016](https://downloads.usda.library.cornell.edu/usda-esmis/files/k3569432s/kh04dr985/cc08hh91c/CropProdSu-01-12-2016.pdf))

Minimum consumption levels are based on the 2007 Renewable Fuel Standard (RFS). The RFS has been updated several times since its initial implementation in 2007. Most recently (December 7th, 2021), EPA proposed total volume requirements for years 2020, 2021, and 2022 of 17.13, 18.52, and 20.77 billion gallons, respectively. These proposed standards have not been implemented, though, so we use the [most recent finalized fuel standards](https://www.epa.gov/renewable-fuel-standard-program/renewable-fuel-annual-standards). Historical ethanol growth projections in the RFS were based partially on the expectation of a growing cellulosic ethanol industry, which has not come to fruition. Due to this and other factors, RFS targets have not been met in recent years, as reported by the Congressional Research Service [(CRS)](https://sgp.fas.org/crs/misc/R43325.pdf). As a result of this noncompliance and the OEO's goals of modeling deep decarbonization, our modeled minimum ethanol use constraint stops in 2030.


The table below represents the minimum total ethanol use and reflect the latest standards.


| Year | Minimum total ethanol use | Maximum total ethanol use (land constraint) |
| ---- | ------------------------- | ------------------------------------------- |
| 2020 | 1598 | 3689 |
| 2025 | 1611 | 3689 |
| 2030 | 1611 | 3689 |
| 2035 |  | 3689 |
| 2040 |  | 3689 |
| 2045 |  | 3689 |
| 2050 | | 3689 |

These values are based on a volumetric energy density of 80.2 MJ per gallon.

### HEFA fuels
Oils and fats can be processed via hydroprocessing to produce hydroprocessed esters and fatty acids (HEFA) in the form of bio-jetfuel and biodiesel. In the OEO database, we assume that soybeans are the input feedstock to the "HEFA" process, and that bio-jetfuel (BIOJTF) or biodiesel (BIODSL) can be produced. Once produced, bio-jetfuel can be blended into conventional jet fuel up to 50\% based on an existing ASTM standard. More information on HEFA fuels is available from [EIA (2015)](https://www.eia.gov/todayinenergy/detail.php?id=23692).

Efficiency data for the HEFA process is drawn from [de Jong (2018)](https://dspace.library.uu.nl/handle/1874/364514), while emissions data is drawn from [GREET (2018)](https://greet.es.anl.gov/). Data on capital costs as well as fixed and variable O&M costs are drawn from a range of sources including [de Jong (2018)](https://dspace.library.uu.nl/handle/1874/364514), [Wormslev et al. (2016)](https://www.nordicenergy.org/wp-content/uploads/2016/09/FULLTEXT_Sustainable_Jet_Fuel_for_Aviation.pdf), [Pavlenko et al. (2019)](https://theicct.org/publication/the-cost-of-supporting-alternative-jet-fuels-in-the-european-union/) and [Bann et al. (2017)](https://www.sciencedirect.com/science/article/abs/pii/S0960852416316911).

### Biodiesel from transesterification
According to [ScienceDirect (2021)](https://www.sciencedirect.com/topics/engineering/transesterification), transesterification is a process in which triacylglycerides from a variety of feedstock such as nonedible oil seeds, vegetable oils, animal fats or tallow, waste cooking oil, and microbial lipids or single cell oil (from algae, oleaginous yeast, filamentous fungi and bacteria) are converted into fatty acid methyl esters (biodiesel) in the presence of alcohol (methanol or ethanol).

In the OEO database, the "TRANSEST" process converts soybeans (SOY) into biodiesel (BIODSL) via the transesterification process. Emissions data associated with the HEFA process is drawn from [GREET (2018)](https://greet.es.anl.gov/). Data on capital cost as well as fixed and variable O&M cost and efficiency data is based on [Huang et al. (2016)](https://onlinelibrary.wiley.com/doi/full/10.1002/bbb.1640).

### Biomass to Fischer-Tropsch fuels
Biomass feedstocks can also be converted to biodiesel and bio-jetfuel via the Fischer-Tropsch (FT) process. Efficiency and emissions data associated with the FT process (FISCH_TROP) are drawn from [GREET (2018)](https://greet.es.anl.gov/). Data on capital costs as well as fixed and variable O&M costs are drawn from a range of sources including [de Jong (2018)](https://dspace.library.uu.nl/handle/1874/364514), [Wormslev et al. (2016)](https://www.nordicenergy.org/wp-content/uploads/2016/09/FULLTEXT_Sustainable_Jet_Fuel_for_Aviation.pdf), [Pavlenko et al. (2019)](https://theicct.org/publication/the-cost-of-supporting-alternative-jet-fuels-in-the-european-union/) and [Bann et al. (2017)](https://www.sciencedirect.com/science/article/abs/pii/S0960852416316911). We assume that the process can accept woody biomass as input. 

# Technology/commodity description lookup tool <a class="anchor" id="description_look_up"></a>
Use the tool below to search for any key words that may describe a technology or commodity of interest (e.g. heating, cooling). The tool provides a list of all the technologies or commodities in the database that may be relevant to the query.


In [4]:
w = widgets.Text(value='HEFA')
display(w)
def f(w):
    if len(w)>0:
        df1 = pd.read_sql("SELECT * FROM commodities WHERE comm_desc LIKE '%" + w + "%'", con)
        df1['desc'] = df1['comm_desc'].str.replace('#','').str.strip()
        df1['comm_tech'] = df1['comm_name']
        df1['type'] = 'commodity'

        df2 = pd.read_sql("SELECT * FROM technologies WHERE tech_desc LIKE '%" + w + "%'", con)
        df2['desc'] = df2['tech_desc'].str.replace('#','').str.strip()
        df2['comm_tech'] = df2['tech']
        df2['type'] = 'technology'


        df = pd.concat([df1[['comm_tech','type','desc']], df2[['comm_tech','type','desc']]])
        
        if len(df)>0:
            display(HTML(tabulate.tabulate(df.set_index('comm_tech'),['technology/commodity','type','description'],stralign='left',tablefmt='html')))
        else:
            print('')
    else:
        print('')
            


out = widgets.interactive_output(f, {'w': w})
display(out)

Text(value='HEFA')

Output()

# Network diagram lookup tool  <a class="anchor" id="network_look_up"></a>
Use the [description lookup tool](#description_look_up) above to identify specific commodity or technology names. Type the name in the box below to generate a corresponding network diagram for that commodity or technology. The slider can be used to view different upstream levels of the network diagram.

In [5]:
w = widgets.Text(value='HEFA')
display(w)
def f(w):
    if len(w)>0:
        
        df1 = pd.read_sql("SELECT comm_name, comm_desc FROM commodities WHERE comm_name='" + w + "'", con)
        df1['desc'] = df1['comm_desc'].str.replace('#','').str.strip()
        df1['comm_tech'] = df1['comm_name']
        df1['type'] = 'commodity'

        df2 = pd.read_sql("SELECT * FROM technologies WHERE tech='" + w + "'", con)
        df2['desc'] = df2['tech_desc'].str.replace('#','').str.strip()
        df2['comm_tech'] = df2['tech']
        df2['type'] = 'technology'

        df = pd.concat([df1[['comm_tech','type','desc']], df2[['comm_tech','type','desc']]])

    
        if len(df)>0:
            def show_desc(level):
                display(Markdown(df['desc'][0]))
                final_dem = df['comm_tech'][0]
                df_graph = return_flowd_table(final_dem,level)
                args = create_args_flowd(df_graph)
                colors, quick_run_dot_fmt = return_format_colors()
                args.update(colors)
                #o_str = 'rankdir = "LR" ;'
                #r_str = 'rankdir = "LR" ; \n\t size="8,8";'
                #quick_run_dot_fmt = quick_run_dot_fmt.replace(o_str, r_str)
                dot_graph = quick_run_dot_fmt % args
                display(graphviz.Source(dot_graph))
            w2 = widgets.IntSlider(value=1,min=0,max=10,step=1,description='Level:',disabled=False,continuous_update=True,orientation='horizontal',readout=True,readout_format='d')
            w = widgets.interactive(show_desc, level=w2)
            controls_rows(w)
        else:
            print('')

    else:
        print('')
            


out = widgets.interactive_output(f, {'w': w})
display(out)

Text(value='HEFA')

Output()